In [0]:
import nltk
nltk.download('stopwords')
nltk.download('brown')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [0]:
!pip -q install summa
from summa import summarizer

     |████████████████████████████████| 61kB 1.9MB/s 


In [0]:
!pip install bert-extractive-summarizer

In [0]:
from summarizer import Summarizer

body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

model = Summarizer()


In [0]:
result = model(body, min_length=60)
full = ''.join(result)
print(full)

The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal. The building sold fairly quickly after being publicly placed on the market only two months ago. The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.


In [0]:
import pandas as pd
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
from operator import itemgetter
import re

In [0]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd 'drive/My Drive/SI630/emailproject'

/content/drive/My Drive/SI630/emailproject


In [0]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [0]:
train.head()

,Subject,content
0,new notice from transwestern pipeline co.,transwestern pipeline co. posted new notice(s)...
1,market alert: nasdaq composite ends down 3.7%,__________________________________\nmarket ale...
2,new notice from transwestern pipeline co.,transwestern pipeline co. posted new notice(s)...
3,ken lay and jeff skilling on cnnfn,ken lay and jeff skilling were interviewed on ...
4,new notice from transwestern pipeline co.,transwestern pipeline co. posted new notice(s)...


In [0]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

def sent_similartity(s1,s2,remove_stopwords=True):
  stop_words=stopwords.words('english')
  s1=tokenizer.tokenize(s1)
  s2=tokenizer.tokenize(s2)
  if remove_stopwords:
    s1=[w for w in s1 if w not in stop_words]
    s2=[w for w in s2 if w not in stop_words]
  words= list(set(s1+s2))
  v1=np.zeros(len(words))
  v2=np.zeros(len(words))
  for w in s1:
    v1[words.index(w)]+=1
  for w in s2:
    v2[words.index(w)]+=1
  return 1 - cosine_distance(v1, v2)

In [0]:
def sim_matrix(sents,remove_stopwords=True):
  n_sents=len(sents)
  M= np.ones((n_sents,n_sents))
  for i in range(n_sents):
    for j in range(n_sents):
      if i == j: 
        continue
      else:
        M[i][j]=sent_similartity(sents[i],sents[j],remove_stopwords)
  return M

In [0]:
#Citation: PageRank
#https://en.wikipedia.org/wiki/PageRank
def pagerank(M, num_iterations=10, d=0.85):
    N = M.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    M_hat = (d * M + (1 - d) / N)
    for i in range(num_iterations):
        v = M_hat @ v
    return v

In [0]:
def TextRank(doc, max_n_sent=10, top_n=1, remove_stopwords=True):
  doc=doc.strip()
  sents=re.split('[?.!:]', doc)
  try: sents=sents[:max_n_sent]
  except: pass
  sents=[s for s in sents if len(s)>10]
  M=sim_matrix(sents,remove_stopwords)
  try:
    Rank=pagerank(M)
    idx=(-Rank.flatten()).argsort()[:top_n]
    result=[sents[i] for i in idx]
    result='.'.join(result)
  except:
    result='NA'
  return result.replace('\n','')

In [0]:
TextRank('Dear xx: Attached you will find an excel document. The outages contained within are forced or rescheduled outages. Your daily delivery will still contain these outages.In addition to the two excel documents, there is a dbf file that is formatted like your daily deliveries you receive nightly. This will enable you to load the data into your regular database. Any questions please let me know.')

' Your daily delivery will still contain these outages'

In [0]:
#LEAD-2
def LEAD2(data):
  data=data.strip()
  RE='^.*?:'
  data=re.sub(RE,'', data)
  data=data.strip()
  data=re.split('[?.,!:]', data)
  result=data[0]
  try: result=result+data[1]
  except: pass
  return result.replace('\n','')

In [0]:
def SUMMA(data):
  try: result=summarizer.summarize(data).partition('.')[0]
  except: result="NA"
  return result

In [0]:

test['result1']=test['content'].apply(lambda x:LEAD2(x))

In [0]:
test['result2']=test['content'].apply(lambda x:SUMMA(x))

In [0]:
test['result3']=test['content'].apply(lambda x:TextRank(x))

In [0]:
test.to_csv('results.csv')

In [0]:
results=pd.read_csv('results.csv')

In [0]:
from nltk.translate.bleu_score import sentence_bleu
def evaluation(ref,candi):
  ref=[tokenizer.tokenize(ref)]
  candi=tokenizer.tokenize(candi)
  bleu=sentence_bleu(ref, candi)
  return bleu

In [0]:
origin=test.Subject
result1=test['result1']#LEAD2
result2=test['result2']#SUMMA
result3=test['result3']#TEXTRANK

In [0]:
score1=[]
score2=[]
score3=[]
def compute_score(origin,data):
  score=[]
  for i in range(len(origin)):
    try:score.append(evaluation(origin[i],data[i]))
    except:score.append(0)
  return score

In [0]:
score1=compute_score(origin,result1)
score2=compute_score(origin,result2)
score3=compute_score(origin,result3)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
from statistics import mean
print('LEAD2 BLEU Score:',mean(score1))#LEAD2
print('SUMMA BLEU Score:',mean(score2))#SUMMA
print('TEXTRANK BLEU Score:',mean(score3))#TEXTRANK

LEAD2 BLEU Score: 0.22323359040730634
SUMMA BLEU Score: 0.23601891889280927
TEXTRANK BLEU Score: 0.2521238135848392


In [0]:
#DEMO
print("///////////////////////////////////////////////////////")
print("Type in your email:")
email_content=input()
print("///////////////////////////////////////////////////////")
print("Type in your subject line:")
subject=input()
print("///////////////////////////////////////////////////////")
print("subject1:",LEAD2(email_content))
print("Score:",evaluation(subject,LEAD2(email_content)))
print("subject2:",SUMMA(email_content))
print("Score:",evaluation(subject,SUMMA(email_content)))
print("subject3:",TextRank(email_content))
print("Score:",evaluation(subject,TextRank(email_content)))

///////////////////////////////////////////////////////
Type in your email:
Attached you will find an excel document. The outages contained within are forced or rescheduled outages. Your daily delivery will still contain these outages.In addition to the two excel documents, there is a dbf file that is formatted like your daily deliveries you receive nightly. This will enable you to load the data into your regular database. Any questions please let me know.
///////////////////////////////////////////////////////
Type in your subject line:
Original Subject & WSCC (Diablo Canyon)  MAAC (Indian River)  MAPP (Gerald Gentleman) NPCC  (Ravenswood)
///////////////////////////////////////////////////////
subject1: Attached you will find an excel document The outages contained within are forced or rescheduled outages
Score: 0
subject2: Your daily delivery will still contain these outages
Score: 0
subject3:  Your daily delivery will still contain these outages
Score: 0


In [0]:
testsize=10
r1,r2,r3,r4=[],[],[],[]
print("0=poor,1=informative or succinct,2=informative and succinct\n")
sample=test.sample(testsize,random_state=2)
for i in range(testsize):
  print("/////////////////Content://////////////////////////////\n",sample.content.iloc[i][:500])
  print("///////////////////////////////////////////////////////")
  print("************Subject line 1:*********\n",sample.result1.iloc[i])
  print("***************************************************")
  print("Score:")
  r1.append(int(input()))
  print("************Subject line 2:*********\n",sample.result2.iloc[i])
  print("***************************************************")
  print("Score:")
  r2.append(int(input()))
  print("************Subject line 3:*********\n",sample.result3.iloc[i])
  print("***************************************************")
  print("Score:")
  r3.append(int(input()))
  print("************Subject line 4:*********\n",sample.Subject.iloc[i])
  print("***************************************************")
  print("Score:")
  r4.append(int(input()))
  
print("final result model1:",sum(r1)/testsize)
print("final result model2:",sum(r2)/testsize)
print("final result model3:",sum(r3)/testsize)
print("final result model3:",sum(r4)/testsize)

0=poor,1=informative or succinct,2=informative and succinct

/////////////////Content://////////////////////////////
 dear board task force:

please disregard the previous note with the same subject.

at meetings with eei recently, they asked jim buccigross, jim templeton and i
if we could send them (larry brown) a letter indicating that the gisb
organization did not intend to impede the progress of eei/cubr in developing
uniform business rules, and further that we would support such a product if a
new organization were created.  attached is a draft letter, carefully worded
-- and probably not everything that t
///////////////////////////////////////////////////////
************Subject line 1:*********
 please disregard the previous note with the same subjectat meetings with eei recently
***************************************************
Score:
1
************Subject line 2:*********
 attached is a draft letter, carefully worded
sent to you previously, and white paper, a strawman, and 

In [0]:
final result model1: 0.5
final result model2: 0.4
final result model3: 0.6
final result model3: 1.1

final result model1: 1.1
final result model2: 1.1
final result model3: 1.3
final result model3: 1.5